In [1]:
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
df = pd.read_csv("/content/crypto_prices_180days.csv")
df = df.dropna().reset_index(drop=True)

In [3]:
# Using only close price
y = df["close"].values

In [4]:
# Train/val/test split
N = len(y)
n_train = int(N * 0.7)
n_val   = int(N * 0.15)
n_test  = N - n_train - n_val

y_train, y_val, y_test = y[:n_train], y[n_train:n_train+n_val], y[n_train+n_val:]

In [5]:
# Scaling
scaler = MinMaxScaler()
y_train_2d = y_train.reshape(-1, 1)
scaler.fit(y_train_2d)

y_train_scaled = scaler.transform(y_train_2d).ravel()
y_val_scaled   = scaler.transform(y_val.reshape(-1, 1)).ravel()
y_test_scaled  = scaler.transform(y_test.reshape(-1, 1)).ravel()

In [6]:
# Convert to torch tensors
train_ds = TensorDataset(torch.arange(len(y_train)),
                         torch.tensor(y_train, dtype=torch.float32),
                         torch.tensor(y_train_scaled, dtype=torch.float32))
val_ds   = TensorDataset(torch.arange(len(y_val)),
                         torch.tensor(y_val, dtype=torch.float32),
                         torch.tensor(y_val_scaled, dtype=torch.float32))
test_ds  = TensorDataset(torch.arange(len(y_test)),
                         torch.tensor(y_test, dtype=torch.float32),
                         torch.tensor(y_test_scaled, dtype=torch.float32))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

In [7]:
#Model Definition (Mini-DMMV style)
class TrendBranch(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

In [8]:
class SeasonalBranch(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

In [9]:
class FusionHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.wg = nn.Parameter(torch.tensor(0.0))
    def forward(self, trend_out, season_out):
        g = torch.sigmoid(self.wg)
        return g * season_out + (1 - g) * trend_out

In [10]:
trend_branch = TrendBranch().to(DEVICE)
season_branch = SeasonalBranch().to(DEVICE)
fusion_head = FusionHead().to(DEVICE)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(
    list(trend_branch.parameters()) +
    list(season_branch.parameters()) +
    list(fusion_head.parameters()),
    lr=1e-3
)

In [11]:
# 5. Training Loop
def train_epoch(loader):
    trend_branch.train()
    season_branch.train()
    fusion_head.train()
    total_loss = 0.0
    for idx, y_orig, y_scaled in loader:
        inp = y_scaled.unsqueeze(-1).to(DEVICE)
        y_scaled = y_scaled.unsqueeze(-1).to(DEVICE)

        trend_pred = trend_branch(inp)
        season_pred = season_branch(inp)
        pred = fusion_head(trend_pred, season_pred)

        loss = loss_fn(pred, y_scaled)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inp.size(0)
    return total_loss / len(loader.dataset)

In [12]:
#Evaluation
def eval_model(loader):
    trend_branch.eval()
    season_branch.eval()
    fusion_head.eval()
    preds_scaled, trues_scaled, trues_orig = [], [], []
    with torch.no_grad():
        for idx, y_orig, y_scaled in loader:
            inp = y_scaled.unsqueeze(-1).to(DEVICE)
            y_scaled = y_scaled.unsqueeze(-1).to(DEVICE)

            trend_pred = trend_branch(inp)
            season_pred = season_branch(inp)
            pred = fusion_head(trend_pred, season_pred)

            preds_scaled.extend(pred.cpu().numpy().reshape(-1,1))
            trues_scaled.extend(y_scaled.cpu().numpy().reshape(-1,1))
            trues_orig.extend(y_orig.numpy().reshape(-1,1))

    preds_scaled = np.array(preds_scaled).reshape(-1,1)
    trues_scaled = np.array(trues_scaled).reshape(-1,1)
    trues_orig   = np.array(trues_orig).reshape(-1,1)

    preds_rescaled = scaler.inverse_transform(preds_scaled).ravel()
    trues_rescaled = scaler.inverse_transform(trues_scaled).ravel()

    mae = mean_absolute_error(trues_orig.ravel(), preds_rescaled)
    rmse = math.sqrt(mean_squared_error(trues_orig.ravel(), preds_rescaled))
    return mae, rmse, preds_rescaled, trues_orig.ravel()

In [13]:
#Training
for epoch in range(20):
    train_loss = train_epoch(train_loader)
    val_mae, val_rmse, _, _ = eval_model(val_loader)
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.6f}, Val MAE={val_mae:.4f}, Val RMSE={val_rmse:.4f}")

Epoch 1: Train Loss=0.133552, Val MAE=23224.8008, Val RMSE=23226.4456
Epoch 2: Train Loss=0.083952, Val MAE=21871.4453, Val RMSE=21872.5428
Epoch 3: Train Loss=0.049592, Val MAE=14566.7998, Val RMSE=14567.7990
Epoch 4: Train Loss=0.027483, Val MAE=11188.5098, Val RMSE=11189.2370
Epoch 5: Train Loss=0.013274, Val MAE=7311.1221, Val RMSE=7311.6001
Epoch 6: Train Loss=0.005498, Val MAE=4360.8755, Val RMSE=4361.2113
Epoch 7: Train Loss=0.001839, Val MAE=2304.2878, Val RMSE=2304.5285
Epoch 8: Train Loss=0.000505, Val MAE=1094.1233, Val RMSE=1094.3183
Epoch 9: Train Loss=0.000118, Val MAE=408.8864, Val RMSE=409.1281
Epoch 10: Train Loss=0.000033, Val MAE=131.5609, Val RMSE=132.0636
Epoch 11: Train Loss=0.000019, Val MAE=8.8776, Val RMSE=12.4291
Epoch 12: Train Loss=0.000016, Val MAE=10.8352, Val RMSE=15.6935
Epoch 13: Train Loss=0.000014, Val MAE=10.5615, Val RMSE=15.4274
Epoch 14: Train Loss=0.000012, Val MAE=13.0779, Val RMSE=13.4932
Epoch 15: Train Loss=0.000010, Val MAE=22.5067, Val RMSE

In [14]:
#Test Evaluation
test_mae, test_rmse, y_pred, y_true = eval_model(test_loader)
print("Test MAE:", test_mae, "Test RMSE:", test_rmse)

Test MAE: 37.81850814819336 Test RMSE: 37.818517083235825
